In [1]:
# Imports
from os import listdir
import numpy as np
import tensorflow as tf
import random as python_random

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
#from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [12]:
np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

In [3]:
print("Versão do TensorFlow: "+tf.__version__)

Versão do TensorFlow: 2.3.0


In [4]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [5]:
!ls

Leishmania_Tensorflow.ipynb  Leishmania_TreinamentoSemAzul
Leishmania_Teste	     README.md
Leishmania_TesteSemAzul      Xception_Leshmania.ipynb
Leishmania_Treinamento	     modelo_leishmania.h5


In [6]:
#import cv2
#import os

#def filtroRGB(src,r,g,b):
#	if r == 0:
#		src[:,:,2] = 0    #elimina o vermelho
#	if g == 0:
#		src[:,:,1] = 0    #elimina o verde
#	if b == 0:
#		src[:,:,0] = 0    #elimina o azul

#def getImages():
#	caminhos = [os.path.join("Leishmania_Treinamento/validacao/Negativo", f) for f in os.listdir("Leishmania_Treinamento/validacao/Negativo")]
#	images =[]
#	i=1
#	for caminhoImagem in caminhos:
#		imagemCarregada = cv2.imread(caminhoImagem)
#		filtroRGB(imagemCarregada,1,1,0)
#		#cv2.imshow(str(i)+".jpg",imagemCarregada)
#		cv2.imwrite("Leishmania_TreinamentoSemAzul/validacao/Negativo/" + str(i) + ".jpg",imagemCarregada)
#		print(str(i) + ".jpg")
#		i=i+1
#		cv2.waitKey(10)

#getImages()

In [13]:
# Inicialização do modelo
classificador = Sequential()

tamanho_imagem = (224, 224)
# Carregamento da arquitetura
transfer = Xception(include_top = False, input_shape = (tamanho_imagem[0],tamanho_imagem[1],3),classifier_activation="softmax")

for camada in transfer.layers:
  camada.trainable = True

classificador.add(transfer)

# Saída do modelo
classificador.add(Flatten())
classificador.add(Dense(1, activation = 'sigmoid'))

classificador.compile(optimizer = Adam(learning_rate = 0.00001),
               loss = 'binary_crossentropy', metrics = ['accuracy'])

batch = 3

# Nenhuma forma de Augmentation
# Rescale é apenas normalização
datagen = ImageDataGenerator(rescale=1./255)



grupo_treinamento = datagen.flow_from_directory('Leishmania_TreinamentoSemAzul/treinamento', 
                                                target_size=tamanho_imagem, 
                                                batch_size=batch, 
                                                class_mode='binary')

grupo_validacao = datagen.flow_from_directory('Leishmania_TreinamentoSemAzul/validacao', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')




Found 320 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [14]:
classificador.fit(grupo_treinamento, epochs=15, validation_data=grupo_validacao)

Epoch 1/15
107/107 [==============================] - 18s 171ms/step - loss: 0.6409 - accuracy: 0.6219 - val_loss: 0.4940 - val_accuracy: 0.8000
Epoch 2/15
107/107 [==============================] - 18s 171ms/step - loss: 0.4007 - accuracy: 0.8500 - val_loss: 0.3458 - val_accuracy: 0.8625
Epoch 3/15
107/107 [==============================] - 18s 165ms/step - loss: 0.2573 - accuracy: 0.9156 - val_loss: 0.3184 - val_accuracy: 0.8375
Epoch 4/15
107/107 [==============================] - 21s 192ms/step - loss: 0.1574 - accuracy: 0.9594 - val_loss: 0.1519 - val_accuracy: 0.9375
Epoch 5/15
107/107 [==============================] - 26s 246ms/step - loss: 0.0930 - accuracy: 0.9812 - val_loss: 0.1533 - val_accuracy: 0.9375
Epoch 6/15
107/107 [==============================] - 27s 251ms/step - loss: 0.0727 - accuracy: 0.9719 - val_loss: 0.0971 - val_accuracy: 0.9625
Epoch 7/15
107/107 [==============================] - 18s 164ms/step - loss: 0.0467 - accuracy: 0.9844 - val_loss: 0.0624 - val_ac

In [15]:
# Funções

# Preprocessamento das imagens
def carregar_imagem(arquivo):
  img = image.load_img(arquivo, target_size=tamanho_imagem)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = img/255

  return img

# Carregar imagens em uma lista
def predicao_teste(pasta_teste):
  imagens = []
  for pasta in list(reversed(listdir(pasta_teste))):
    for arquivo in listdir(pasta_teste + '/' + pasta):
      imagens.append(carregar_imagem(pasta_teste + pasta + '/' + arquivo))
  
  return imagens

In [16]:
imagens = np.vstack(predicao_teste('Leishmania_TesteSemAzul/'))
respostas_modelo = classificador.predict_classes(imagens)
respostas_teste = datagen.flow_from_directory('Leishmania_TesteSemAzul/', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')
print('Respostas do modelo\n' + str(np.concatenate(respostas_modelo)))
print('Respostas reais\n' + str(respostas_teste.classes))

# Matriz de confusão
matriz_de_confusao = confusion_matrix(respostas_teste.classes, respostas_modelo)

# Metricas
# Marvin pediu especificamente pra gente avaliar os valores de sensibilidade e
# especificidade
TrueNegative, FalsePositive, FalseNegative, TruePositive = matriz_de_confusao.ravel()

# Precisão
print('Precisao ' + str((TruePositive + TrueNegative) / len(respostas_modelo)))

# Sensibilidade
print('Sensibilidade ' + str(TruePositive / (TruePositive + FalseNegative)))

# Especificidade
print('Especificidade ' + str(TrueNegative / (FalsePositive + TrueNegative)))

Found 44 images belonging to 2 classes.
Respostas do modelo
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 1 0 0
 1 1 0 0 1 1 0]
Respostas reais
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1]
Precisao 0.7954545454545454
Sensibilidade 0.6363636363636364
Especificidade 0.9545454545454546
